# Mesa document
https://mesa.readthedocs.io/en/master/tutorials/intro_tutorial.html

#  Sample
https://github.com/projectmesa/mesa/blob/main/examples/wolf_sheep/wolf_sheep/agents.py
https://github.com/projectmesa/mesa/blob/main/examples/wolf_sheep/wolf_sheep/model.py



In [1]:
pip install mesa # Mesaのインストール

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 53.6 MB/s 
     |████████████████████████████████| 64 kB 3.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Define Agent

In [2]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

import numpy as np
import pandas as pd

##############
#
# Client Agent
#
##############
class ClientUser(Agent):
  def __init__(self, unique_id, pos, type, model):
    super().__init__(unique_id, model)
    self.pos = pos
    self.isClient = True
    self.isTarget = False
    self.isHost = False
    self.gain = 0 # dummy
    self.datarate = 0 # dummy
    self.connected = 0 # dummy
    self.demand = 0 # Demand d_{c}(t)
    self.type = type # Application type
    self.allocated = -1 # Allocated host

  def move(self): # Random walk
    possible_steps = self.model.grid.get_neighborhood(
        self.pos,
        moore=True, # includes all 8 surrounding squares
        include_center=True) # include the center cell itself
    new_position = self.random.choice(possible_steps)
    self.model.grid.move_agent(self, new_position)

  def updateDemand(self): # Randomly update demand
    if self.random.random() < 0.1: # 確率 10%
      self.type = self.random.randint(0, 2) # Change application type
    if self.type == 0:
      self.demand = self.random.randint(10, 20) # High rate
    elif self.type == 1:
      self.demand = self.random.randint(5, 10) # Middle rate
    else: # 2
      self.demand = self.random.randint(0, 5) # Low rate

  def allocate2Host(self, host_id):
    self.allocated = host_id

  def step(self):
    self.allocated = -1
    if self.random.random() < 0.1: # 確率 10%
      self.move() # Random walk
    self.updateDemand() # Update demand

##############
#
# Target Agent
#
##############
class Target(Agent):
  def __init__(self, unique_id, pos, model):
    super().__init__(unique_id, model)
    self.pos = pos
    self.isClient = False
    self.isTarget = True
    self.isHost = False
    self.gain = 0 # dummy
    self.datarate = 0 # dummy
    self.connected = 0 # dummy
    self.num_client = 0 # number of clients in cell
    self.demand = 0 # total demand in cell

  def countClient(self): # Count client and demand in cell
    x, y = self.pos
    this_cell = self.model.grid.get_cell_list_contents([self.pos])
    clients = [obj for obj in this_cell if isinstance(obj, ClientUser)]
    self.num_client = len(clients)
    self.demand = 0
    for i in range(self.num_client):
      self.demand += clients[i].demand

  def meetDemand(self, host_id, datarate):
    self.datarate += datarate
    this_cell = self.model.grid.get_cell_list_contents([self.pos])
    clients = [obj for obj in this_cell if isinstance(obj, ClientUser)]
    for i in range(len(clients)):
      if clients[i].allocated == -1:
        clients[i].allocate2Host(host_id)

  def step(self):
    self.datarate = 0 # reset datarate
    self.countClient() # Count client

##############
#
# Host Agent
#
##############
class HostUser(Agent):
  def __init__(self, unique_id, host_id, pos, max_rate, strategy, rad2search, model):
    super().__init__(unique_id, model)
    self.pos = pos
    self.isClient = False
    self.isTarget = False
    self.isHost = True
    self.gain = 0 # g_{h}(t)
    self.datarate = 0 # Total data rate of connected clients
    self.connected = 0 # Number of connected clients
    self.host_id = host_id
    self.max_rate = max_rate # w_{h}
    self.strategy = strategy
    self.rad2search = rad2search

  def getGain(self):
    this_cell = self.model.grid.get_cell_list_contents([self.pos])
    target = [obj for obj in this_cell if isinstance(obj, Target)]
    self.connected = target[0].num_client
    self.datarate = min(target[0].demand - target[0].datarate, self.max_rate)
    target[0].meetDemand(self.host_id, self.datarate)
    if self.strategy == 1:
      self.gain = target[0].demand
    elif self.strategy == 2:
      self.gain = target[0].num_client
    else: # random walk
      self.gain = 1

  def move(self):
    if self.strategy == 1 or self.strategy == 2:
      # Search targets
      searchedTargets = self.model.grid.get_neighbors(self.pos, moore=True, include_center=True, radius=self.rad2search) # Get surrounding all agents
      searchedTargets = [item for item in searchedTargets if item.isTarget == True] # Get Targets only
      max_val = 0
      max_idx = 0
      for i in range (len(searchedTargets)): # 左上，上，右上，左，中心，右，左下，下，右下
        if self.strategy == 1:
          val = searchedTargets[i].demand
        else:
          val = searchedTargets[i].num_client
        if val > max_val:
          max_val = val
          max_idx = i
      new_position = searchedTargets[max_idx].pos
      #print("new position:", new_position)
    else: # random walk
      possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=True)
      new_position = self.random.choice(possible_steps)
    # Move
    self.model.grid.move_agent(self, new_position)

  def step(self):
    self.move()
    self.getGain()


# Define Model

In [3]:
##############
#
# Data Collection
#
##############
def compute_objective(model):
  client_demand = [agent.demand for agent in model.schedule.agents if isinstance(agent, ClientUser)]
  host_datarate = [agent.datarate for agent in model.schedule.agents if isinstance(agent, HostUser)]
  model.objective += sum(host_datarate) / sum(client_demand)
  return model.objective

def compute_gain(model):
  host_gain = [agent.gain for agent in model.schedule.agents if isinstance(agent, HostUser)]
  return host_gain

def compute_datarate(model):
  host_datarate = [agent.datarate for agent in model.schedule.agents if isinstance(agent, HostUser)]
  model.total_datarate += sum(host_datarate)
  return model.total_datarate

def compute_connected(model):
  host_connected = [agent.connected for agent in model.schedule.agents if isinstance(agent, HostUser)]
  model.total_connected += sum(host_connected)
  return model.total_connected

def map_host(model):
  hostMap = []
  host_pos = [agent.pos for agent in model.schedule.agents if isinstance(agent, HostUser)]
  for i in range(len(host_pos)):
    hostMap.append(host_pos[i])
  return hostMap

def map_client(model):
  clientMap = []
  client_pos = [agent.pos for agent in model.schedule.agents if isinstance(agent, ClientUser)]
  client_type = [agent.type for agent in model.schedule.agents if isinstance(agent, ClientUser)]
  client_allocated = [agent.allocated for agent in model.schedule.agents if isinstance(agent, ClientUser)]
  for i in range(len(client_pos)):
    x, y = client_pos[i]
    tmp = [x, y, client_type[i], client_allocated[i]]
    clientMap.append(tmp)
  return clientMap

def heatmap_client(model):
  clientHeatmap = [[0] * model.width for i in range(model.height)]
  target_pos = [agent.pos for agent in model.schedule.agents if isinstance(agent, Target)]
  target_num_client = [agent.num_client for agent in model.schedule.agents if isinstance(agent, Target)]
  for i in range(len(target_num_client)):
    x, y = target_pos[i]
    clientHeatmap[y][x] = target_num_client[i]
  return clientHeatmap

def heatmap_demand(model):
  demandHeatmap = [[0] * model.width for i in range(model.height)]
  target_pos = [agent.pos for agent in model.schedule.agents if isinstance(agent, Target)]
  target_demand = [agent.demand for agent in model.schedule.agents if isinstance(agent, Target)]
  for i in range(len(target_demand)):
    x, y = target_pos[i]
    demandHeatmap[y][x] = target_demand[i]
  return demandHeatmap

##############
#
# Model
#
##############
class D2dGameModel(Model):
  def __init__(self, num_client, num_host, width, height, max_type, max_rate, host_strategy, rad2search, seed=None):
    self.current_id = 0
    self.num_client = num_client
    self.num_host = num_host
    self.width = width
    self.height = height
    self.max_type = max_type
    self.max_rate = max_rate
    self.host_strategy = host_strategy
    self.rad2search = rad2search
    self.clientMap = [[0] * width for i in range(height)]
    self.demandMap = [[0] * width for i in range(height)]
    self.objective = 0 # Demand satisfaction
    self.total_datarate = 0 # Total datarate
    self.total_connected = 0 # Total connected
    self.grid = MultiGrid(width, height, True)
    self.schedule = RandomActivation(self)
    print("Client:", num_client, "Host:", num_host, "Max.type:", max_type, "Max.rate:", max_rate, "Strategy:", host_strategy, "Rad2Search:", rad2search)

    # Create ClientUser agents
    for i in range(self.num_client):
      x = self.random.randrange(self.grid.width)
      y = self.random.randrange(self.grid.height)
      rnd_type = self.random.randint(0, self.max_type)
      client = ClientUser(self.next_id(), (x, y), rnd_type, self)
      self.clientMap[y][x] += 1
      self.grid.place_agent(client, (x, y))
      self.schedule.add(client)

    # Create Target agents
    for i in range(self.grid.width):
      for j in range(self.grid.height):
        x = i
        y = j
        target = Target(self.next_id(), (x, y), self)
        self.grid.place_agent(target, (x, y))
        self.schedule.add(target)

    # Create HostUser agents
    hostId = 0
    for i in range(self.num_host):
      x = self.random.randrange(self.grid.width)
      y = self.random.randrange(self.grid.height)
      rnd_rate = self.random.randint(0, self.max_rate)
      host = HostUser(self.next_id(), hostId, (x, y), rnd_rate, self.host_strategy, self.rad2search, self)
      self.grid.place_agent(host, (x, y))
      self.schedule.add(host)
      hostId += 1

    # Create DataCollector
    self.datacollector = DataCollector(
      model_reporters = {
          "Objective": compute_objective,
          "Gain": compute_gain,
          "DataRate": compute_datarate,
          "Connected": compute_connected,
          "MapHost": map_host,
          "MapClient": map_client,
          "HeatmapClient": heatmap_client,
          "HeatmapDemand": heatmap_demand,
      },
      agent_reporters = {
          "Pos": lambda a: a.pos,
          "X": lambda a: a.pos[0],
          "Y": lambda a: a.pos[1],
          "Gain": lambda a: [a.isHost, a.gain, a.datarate, a.connected],
      },
    )

    self.running = True

  def updateMap(self):
    self.clientMap = [[0] * self.width for i in range(self.height)]
    self.demandMap = [[0] * self.width for i in range(self.height)]
    targets = [obj for obj in self.schedule.agents if isinstance(obj, Target)]
    for i in range(len(targets)):
      x, y = targets[i].pos
      self.clientMap[y][x] = targets[i].num_client
      self.demandMap[y][x] = targets[i].demand

  def drawMap(self) -> None:
    fig = plt.figure(figsize=(12, 3))
    # Draw heatmaps
    ax1 = fig.add_subplot(1, 3, 1)
    self.text = ax1.set_title(f"Number of clients (t={self.schedule.time:03d})")
    im1 = ax1.imshow(self.clientMap, interpolation='nearest', cmap='Blues', vmin=0, vmax=10)
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im1, cax=cax1)

    ax2 = fig.add_subplot(1, 3, 2)
    self.text = ax2.set_title(f"Demand (t={self.schedule.time:03d})")
    im2 = ax2.imshow(self.demandMap, interpolation='nearest', cmap='Oranges', vmin=0, vmax=50)
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im2, cax=cax2)

    # Draw hosts
    ax3 = fig.add_subplot(1, 3, 3)
    ax3.set_xlim(0, self.width)
    ax3.set_ylim(self.height, 0)
    ax3.set_xticks([0.5, self.width-0.5]) 
    ax3.set_yticks([0.5, self.height-0.5]) 
    ax3.set_xticklabels([0, self.width-1])
    ax3.set_yticklabels([0, self.height-1])
    ax3.set_aspect("equal")
    self.text = ax3.set_title(f"Host (t={self.schedule.time:03d})")
    hosts = [obj for obj in self.schedule.agents if isinstance(obj, HostUser)]
    for i in range (len(hosts)):
      x, y = hosts[i].pos
      x += 0.5
      y += 0.5
      scat = ax3.scatter(x, y, c="red", s=10, zorder=2)
      c = patches.Circle(xy=(x, y), radius=self.rad2search, ec="lightgrey", fc="None", linestyle="--", zorder=1)
      ax3.add_patch(c)
    plt.tight_layout()
    plt.show()

  def step(self):
    self.schedule.step()
    self.datacollector.collect(self)
    self.updateMap() # Update client/demand heatmap
    self.drawMap() # Draw maps at each step


# Single run (Time-series)

In [ ]:
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML 
import csv

##############
#
# Run
#
##############

# Define parameters
Width = 20 # X cells
Height = 20 # Y cells
NumHost = 25 # Number of hosts
NumClient = 500 # Number of clients
MaxType = 2 # Max. type of client
MaxRate = 50 # Max. of host datarate
HostStrategy = 3 # 1: Rate-based, 2: Target size-based, else: Random walk
Rad2Search = 1 # Radius to search targets
NumSteps = 100 # Time steps
Seed = 1 # Seed

model = D2dGameModel(NumClient, NumHost, Width, Height, MaxType, MaxRate, HostStrategy, Rad2Search)
for i in range(NumSteps):
  model.step() # Proceed step

modelData = model.datacollector.get_model_vars_dataframe()
agentData = model.datacollector.get_agent_vars_dataframe()

# Draw maps

In [ ]:
# Count Multiindex of pandas
numStep = len(agentData.groupby(level=0).size())
numAgent = len(agentData.groupby(level=1).size())

# Client heatmap
fig1 = plt.figure() # figsize=(10, 4)
ax1 = fig1.add_subplot(1, 1, 1)
ims1 = []
for t in range(numStep):
  im1 = ax1.imshow(modelData["HeatmapClient"].at[t], interpolation='nearest', cmap='Blues')
  divider1 = make_axes_locatable(ax1)
  cax1 = divider1.append_axes("right", size="5%", pad=0.1)
  plt.colorbar(im1, cax=cax1)
  ims1.append([im1])
  plt.savefig("heatmap_client%d.png" %(t+1))
  with open('heatmap_client%d.csv' %(t+1), 'w') as f: # Write CSV
    writer = csv.writer(f)
    writer.writerows(modelData["HeatmapClient"].at[t])  
#ani1 = animation.ArtistAnimation(fig1, ims1, interval=100, blit=True, repeat_delay=1000)
#ani1.save("anim_heatmap_client.gif", writer='pillow')
#HTML(ani1.to_jshtml())

# Demand heatmap
fig2 = plt.figure() # figsize=(10, 4)
ax2 = fig2.add_subplot(1, 1, 1)
ims2 = []
for t in range(numStep):
  im2 = ax2.imshow(modelData["HeatmapDemand"].at[t], interpolation='nearest', cmap='Oranges')
  divider2 = make_axes_locatable(ax2)
  cax2 = divider2.append_axes("right", size="5%", pad=0.1)
  plt.colorbar(im2, cax=cax2)
  ims2.append([im2])
  plt.savefig("heatmap_demand%d.png" %(t+1))
  with open('heatmap_demand%d.csv' %(t+1), 'w') as f: # Write CSV
    writer = csv.writer(f)
    writer.writerows(modelData["HeatmapDemand"].at[t])  
#ani2 = animation.ArtistAnimation(fig2, ims2, interval=100, blit=True, repeat_delay=1000)
#ani2.save("anim_heatmap_demand.gif", writer='pillow')
#HTML(ani2.to_jshtml())

# Host map
fig3 = plt.figure() # figsize=(10, 4)
ax3 = fig3.add_subplot(1, 1, 1)
ims3 = []
for t in range(numStep):
  x = []
  y = []
  x.clear()
  y.clear()
  f = open('host%d.csv' %(t+1), 'w')
  f.close()
  for i in range (len(modelData["MapHost"].at[t])):
    x.append(modelData["MapHost"].at[t][i][0])
    y.append(modelData["MapHost"].at[t][i][1])
    with open('host%d.csv' %(t+1), 'a') as f: # Write CSV
      writer = csv.writer(f)
      writer.writerow([modelData["MapHost"].at[t][i][0], modelData["MapHost"].at[t][i][1]])
  im3 = ax3.scatter(x, y, color="slateblue")
  ims3.append([im3])
  plt.savefig("map_host%d.png" %(t+1))
#ani3 = animation.ArtistAnimation(fig3, ims3, interval=100, blit=True, repeat_delay=1000)
#ani3.save("anim_host.gif", writer='pillow')
#HTML(ani3.to_jshtml())

# Client map
fig4 = plt.figure() # figsize=(10, 4)
ax4 = fig4.add_subplot(1, 1, 1)
ims4 = []
for t in range(numStep):
  x = []
  y = []
  x.clear()
  y.clear()
  f = open('client%d.csv' %(t+1), 'w')
  f.close()
  for i in range (len(modelData["MapClient"].at[t])):
    x.append(modelData["MapClient"].at[t][i][0])
    y.append(modelData["MapClient"].at[t][i][1])
    with open('client%d.csv' %(t+1), 'a') as f: # Write CSV
      writer = csv.writer(f)
      writer.writerow([modelData["MapClient"].at[t][i][0], modelData["MapClient"].at[t][i][1], modelData["MapClient"].at[t][i][2], modelData["MapClient"].at[t][i][3]])
  im4 = ax4.scatter(x, y, color="slateblue")
  ims4.append([im4])
  plt.savefig("map_client%d.png" %(t+1))
#ani4 = animation.ArtistAnimation(fig4, ims4, interval=100, blit=True, repeat_delay=1000)
#ani4.save("anim_client.gif", writer='pillow')
#HTML(ani4.to_jshtml())


# Show results

In [ ]:
print(modelData.Objective, modelData.Gain, modelData.DataRate, modelData.Connected)
print(modelData.MapClient, modelData.MapDemand, modelData.MapHost)
print(agentData.Gain, agentData.X, agentData.Y)

x = list(agentData["X"])
y = list(agentData["Y"])
gain = list(agentData["Gain"])

#print(agentData["Gain"])
#print(agentData["Gain"].at[10])
#print(agentData["Gain"].at[10].at[108])


# Batch run (Parameters vs Final result)
https://mesa.readthedocs.io/en/stable/_modules/mesa/batchrunner.html?highlight=mesa.batchrunner#

In [ ]:
from mesa.batchrunner import BatchRunner

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML 
import csv

NumIterations = 20
NumSteps = 100

# num_client, num_host, width, height, max_demand, max_rate, host_strategy, rad2search
fixed_params = {"width": 20, "height": 20, "num_client": 500, "num_host": 25, "max_type": 2, "max_rate": 50, "rad2search": 1}
variable_params = {"host_strategy": range(1, 4, 1)}

batch_run = BatchRunner(D2dGameModel,
                        variable_params,
                        fixed_params,
                        iterations=NumIterations,
                        max_steps=NumSteps,
                        model_reporters = {
                            "Objective": compute_objective,
                            "Gain": compute_gain,
                            "DataRate": compute_datarate,
                            "Connected": compute_connected,
                            "MapHost": map_host,
                            "MapClient": map_client,
                            "HeatmapClient": heatmap_client,
                            "HeatmapDemand": heatmap_demand,
                            },
#                        agent_reporters = {
#                            "Pos": lambda a: a.pos,
#                            "X": lambda a: a.pos[0],
#                            "Y": lambda a: a.pos[1],
#                            "Gain": lambda a: [a.isHost, a.gain, a.datarate, a.connected],
#                            },
                        )
batch_run.run_all()

modelData = batch_run.get_model_vars_dataframe()
modelData.head()
#agentData = batch_run.get_agent_vars_dataframe()
#agentData.head()

In [ ]:
# Time average
N = len(modelData)
strategy = []
objective = []
totalDatarate = []
totalConnected = []
for i in range(N):
  strategy.append(modelData.host_strategy[i])
  objective.append(modelData.Objective[i])
  totalDatarate.append(modelData.DataRate[i]/NumSteps)
  totalConnected.append(modelData.Connected[i]/NumSteps)

strt = []
mean_objective = []
mean_totalDatarate = []
mean_totalConnected = []
stdev_objective = []
stdev_totalDatarate = []
stdev_totalConnected = []
for i in range(1, 4):
  strt.append(i)
  mean_objective.append(modelData[modelData.host_strategy == i].Objective.mean())
  mean_totalDatarate.append(modelData[modelData.host_strategy == i].DataRate.mean()/NumSteps)
  mean_totalConnected.append(modelData[modelData.host_strategy == i].Connected.mean()/NumSteps)
  stdev_objective.append(modelData[modelData.host_strategy == i].Objective.std())
  stdev_totalDatarate.append(modelData[modelData.host_strategy == i].DataRate.std()/NumSteps)
  stdev_totalConnected.append(modelData[modelData.host_strategy == i].Connected.std()/NumSteps)

# Strategy vs Demand satisfaction
print("Strategy vs Demand satisfaction")
print(strt, mean_objective, stdev_objective)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.scatter(strategy, objective, c="pink", alpha=0.5, linewidths="2", edgecolors="red")
ax.errorbar(strt, mean_objective, yerr = stdev_objective, capsize=5, fmt='o', markersize=10, ecolor='gray', markeredgecolor = "black", color='w')
ax.set_xlim(0.5, 3.5)
ax.set_xticks([1, 2, 3]) 
ax.set_xticklabels(["Model 1", "Model 2", "Non-gamified"])
ax.set_xlabel("Strategy")
ax.set_ylabel("Demand satisfaction / period")
plt.savefig("result_satisfaction.png")
plt.show()

# Strategy vs Data rate
print("Strategy vs Data rate")
print(strt, mean_totalDatarate, stdev_totalDatarate)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.scatter(strategy, totalDatarate, c="pink", alpha=0.5, linewidths="2", edgecolors="red")
ax.errorbar(strt, mean_totalDatarate, yerr = stdev_totalDatarate, capsize=5, fmt='o', markersize=10, ecolor='gray', markeredgecolor = "black", color='w')
ax.set_xlim(0.5, 3.5)
ax.set_xticks([1, 2, 3]) 
ax.set_xticklabels(["Model 1", "Model 2", "Non-gamified"])
ax.set_xlabel("Strategy")
ax.set_ylabel("Total data rate / period")
plt.savefig("result_datarate.png")
plt.show()

# Strategy vs Connected
print("Strategy vs Connected")
print(strt, mean_totalConnected, stdev_totalConnected)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.scatter(strategy, totalConnected, c="pink", alpha=0.5, linewidths="2", edgecolors="red")
ax.errorbar(strt, mean_totalConnected, yerr = stdev_totalConnected, capsize=5, fmt='o', markersize=10, ecolor='gray', markeredgecolor = "black", color='w')
ax.set_xlim(0.5, 3.5)
ax.set_xticks([1, 2, 3]) 
ax.set_xticklabels(["Model 1", "Model 2", "Non-gamified"])
ax.set_xlabel("Strategy")
ax.set_ylabel("Connected clients / period")
plt.savefig("result_connected.png")
plt.show()
